In [1]:
import requests
import pandas as pd

In [2]:
API_KEY = ""

# Function to get the subscriber count of a channel
def get_channel_subscriber_count(channel_id):
    url = f"https://www.googleapis.com/youtube/v3/channels?part=statistics&id={channel_id}&key={API_KEY}"
    response = requests.get(url).json()
    try:
        subscriber_count = response['items'][0]['statistics']['subscriberCount']
    except (IndexError, KeyError):
        subscriber_count = 'Not Available'
    return subscriber_count

# Function to get video details using the YouTube Data API
def get_video_details(video_id):
    data_url = f"https://www.googleapis.com/youtube/v3/videos?id={video_id}&part=snippet,statistics&key={API_KEY}"
    response = requests.get(data_url).json()

    snippet = response['items'][0]['snippet']
    statistics = response['items'][0]['statistics']

    video_details = {
        'video_id': video_id,
        'title': snippet['title'],
        'description': snippet['description'],
        'upload_date': snippet['publishedAt'],
        'view_count': statistics['viewCount'],
        'like_count': statistics.get('likeCount', 'Not Available'),
        'comment_count': statistics.get('commentCount', 'Not Available')
    }
    return video_details

# Function to fetch all videos from a channel
def get_channel_videos(channel_id):
    videos = []
    page_token = ""
    while True:
        url = f"https://www.googleapis.com/youtube/v3/search?key={API_KEY}&channelId={channel_id}&part=snippet,id&type=video&maxResults=50&pageToken={page_token}"
        response = requests.get(url).json()

        for item in response['items']:
            video_id = item['id']['videoId']
            video_details = get_video_details(video_id)
            videos.append(video_details)

        page_token = response.get('nextPageToken', '')
        if not page_token:
            break
    return pd.DataFrame(videos)

In [3]:
channels = {
    "Carnage Codm": "UCtuhZ4uWNnvM7BjX2RUeGuA",
    "2noob4u Codm": "UCj_paku3Qlgm8daSGP0jYHg",
    "PrinceRez Codm": "UCGKdHquSjUL3Ge2h-ZhgE2Q",
    "Ednox Codm": "UC68OAM8s1eI998A9HGdXxyw"
}

channel_info = []

for name, channel_id in channels.items():
    print(f"Fetching videos for channel: {name}")
    df = get_channel_videos(channel_id)
    csv_file_path = f"{name.replace(' ', '_')}_Videos.csv"
    df.to_csv(csv_file_path, index=False)
    print(f"DataFrame for {name} successfully exported to {csv_file_path}")

    subscriber_count = get_channel_subscriber_count(channel_id)
    channel_info.append({'Channel Name': name, 'Channel ID': channel_id, 'Subscriber Count': subscriber_count})

channel_info_df = pd.DataFrame(channel_info)
channel_info_df.to_csv('Channel_Subscriber_Counts.csv', index=False)
print("Channel subscriber counts successfully exported to Channel_Subscriber_Counts.csv")

Fetching videos for channel: Carnage Codm
DataFrame for Carnage Codm successfully exported to Carnage_Codm_Videos.csv
Fetching videos for channel: 2noob4u Codm
DataFrame for 2noob4u Codm successfully exported to 2noob4u_Codm_Videos.csv
Fetching videos for channel: PrinceRez Codm
DataFrame for PrinceRez Codm successfully exported to PrinceRez_Codm_Videos.csv
Fetching videos for channel: Ednox Codm
DataFrame for Ednox Codm successfully exported to Ednox_Codm_Videos.csv
Channel subscriber counts successfully exported to Channel_Subscriber_Counts.csv
